# 보고서 전체 리스트

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

base_url = "https://www.kbfg.com/kbresearch/vitamin/reportList.do"

# WebDriver를 사용하여 브라우저 실행
driver = webdriver.Chrome('chromedriver.exe')
driver.get(base_url)

# 페이지가 로드될 때까지 약간의 대기 시간
driver.implicitly_wait(5)


#페이지 넘기기
def move_to_next_page():
    for i in range(1,6):
        page_link = '//*[@id="searchform"]/div/div[4]/span['+str(i)+']/a'
        next_page = driver.find_element_by_xpath(page_link)
        next_page.click()
        parse_current_page()
    turn_page = driver.find_element_by_xpath('//*[@id="searchform"]/div/div[4]/a[3]')
    turn_page.click()
    parse_current_page()
    move_to_next_page()

# 결과를 저장할 리스트 초기화
cates = []
titles = []
links = []
writers = []
dates = []
hits = []
date = []

def parse_current_page():
    # 페이지 소스를 BeautifulSoup으로 파싱
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 파싱
    for report in soup.find("div", class_="listWrap on").find_all("li"):
        link = report.find("a")["href"]
        cate = report.find("span").get_text()
        title = report.find("h3").get_text()
        writer = report.find("dt").get_text()
        date = report.find("dd").get_text()
        hit = report.find("dl").find("span").get_text()
        links.append(link)
        cates.append(cate)
        titles.append(title)
        writers.append(writer)
        dates.append(date)
        hits.append(hit)
    
# 처음 페이지 파싱
parse_current_page()

# 페이지 넘기면서 크롤링
while move_to_next_page():
    pass

# 크롤링이 끝난 후 WebDriver를 종료합니다.
driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="searchform"]/div/div[4]/span[4]/a"}
  (Session info: chrome=115.0.5790.110)


In [ ]:
#크롤링 확인 -> 맨 첫 페이지 겹치는데 나중에 중복제거 하면 됨^^
titles

In [4]:
# 9*52페이지 = 468 (100개가 중복인가봐...)
len(titles)

568

In [5]:
#보고서 리스트 df 저장
import pandas as pd
content_df = pd.DataFrame({"category":cates,
                           "title": titles, 
                           "writer" : writers, 
                           "date" : dates,
                          "hit":hits,
                          "url":links})
content_df

,category,title,writer,date,hit,url
0,산업분석,인공지능 시대 새로운 성장 동력 HBM(고대역폭메모리),심경석,2023-07-24,448,/kbresearch/vitamin/reportView.do?vitaminId=20...
1,디지털,인공지능은 인간지능을 초월할 수 있을까,김진성,2023-06-20,948,/kbresearch/vitamin/reportView.do?vitaminId=20...
2,산업분석,"발전기 없는 발전소, 가상발전소(VPP)",이창우,2023-04-14,1909,/kbresearch/vitamin/reportView.do?vitaminId=20...
3,디지털,해외 대형은행의 온라인 대출중개 플랫폼 활용과 대응 사례,김준산,2022-10-07,6210,/kbresearch/vitamin/reportView.do?vitaminId=20...
4,디지털,트래블룰 적용 의의와 국내 가상자산 업계 동향,박교순,2022-05-30,5031,/kbresearch/vitamin/reportView.do?vitaminId=20...
...,...,...,...,...,...,...
563,부동산,행복주택의 이해 및 추진현황,이종아,2015-01-26,4454,/kbresearch/vitamin/reportView.do?vitaminId=10...
564,자산관리,저금리시대의 저축과 레버리지 투자,김진성,2015-01-21,5201,/kbresearch/vitamin/reportView.do?vitaminId=10...
565,경영환경,O2O 거래의 개념 및 관련 논란의 이해,정훈,2015-01-09,6651,/kbresearch/vitamin/reportView.do?vitaminId=10...
566,부동산,2015년 미국 부동산시장 트렌드 점검,손은경,2015-01-07,3668,/kbresearch/vitamin/reportView.do?vitaminId=10...


In [70]:
content_df.to_csv('경영보고서리스트.csv')

In [ ]:
content_df = pd.read_csv('경영보고서리스트.csv')

In [6]:
content_df_unique = content_df.drop_duplicates()
content_df_unique.to_csv('경영보고서리스트_중복제거.csv')

In [7]:
content_df_unique

,category,title,writer,date,hit,url
0,산업분석,인공지능 시대 새로운 성장 동력 HBM(고대역폭메모리),심경석,2023-07-24,448,/kbresearch/vitamin/reportView.do?vitaminId=20...
1,디지털,인공지능은 인간지능을 초월할 수 있을까,김진성,2023-06-20,948,/kbresearch/vitamin/reportView.do?vitaminId=20...
2,산업분석,"발전기 없는 발전소, 가상발전소(VPP)",이창우,2023-04-14,1909,/kbresearch/vitamin/reportView.do?vitaminId=20...
3,디지털,해외 대형은행의 온라인 대출중개 플랫폼 활용과 대응 사례,김준산,2022-10-07,6210,/kbresearch/vitamin/reportView.do?vitaminId=20...
4,디지털,트래블룰 적용 의의와 국내 가상자산 업계 동향,박교순,2022-05-30,5031,/kbresearch/vitamin/reportView.do?vitaminId=20...
...,...,...,...,...,...,...
563,부동산,행복주택의 이해 및 추진현황,이종아,2015-01-26,4454,/kbresearch/vitamin/reportView.do?vitaminId=10...
564,자산관리,저금리시대의 저축과 레버리지 투자,김진성,2015-01-21,5201,/kbresearch/vitamin/reportView.do?vitaminId=10...
565,경영환경,O2O 거래의 개념 및 관련 논란의 이해,정훈,2015-01-09,6651,/kbresearch/vitamin/reportView.do?vitaminId=10...
566,부동산,2015년 미국 부동산시장 트렌드 점검,손은경,2015-01-07,3668,/kbresearch/vitamin/reportView.do?vitaminId=10...


# pdf 저장 및 추천 리스트

In [8]:
links = content_df_unique['url']
links

0      /kbresearch/vitamin/reportView.do?vitaminId=20...
1      /kbresearch/vitamin/reportView.do?vitaminId=20...
2      /kbresearch/vitamin/reportView.do?vitaminId=20...
3      /kbresearch/vitamin/reportView.do?vitaminId=20...
4      /kbresearch/vitamin/reportView.do?vitaminId=20...
                             ...                        
563    /kbresearch/vitamin/reportView.do?vitaminId=10...
564    /kbresearch/vitamin/reportView.do?vitaminId=10...
565    /kbresearch/vitamin/reportView.do?vitaminId=10...
566    /kbresearch/vitamin/reportView.do?vitaminId=10...
567    /kbresearch/vitamin/reportView.do?vitaminId=10...
Name: url, Length: 469, dtype: object

In [9]:
#pdf 저장 및 추천 보고서 리스트 저장

#추천 리스트
pdftitles = [] 
recommends = []
pdf = []
titles = []

#보고서 url for문 돌면서 pdf 다운로드
for i in links:
    driver = webdriver.Chrome('chromedriver.exe')  # chromedriver 경로를 지정해야 합니다.

    url = 'https://www.kbfg.com/'+i
    driver.get(url)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, 'html.parser', from_encoding = 'utf-8')
    
    #매핑 및 확인용 보고서명 저장
    title = soup.find("div", class_="viewTit").find("h3").get_text()
    titles.append(title)
    
    #추천리스트 저장
    rec_list = []
    for i in soup.find("div", class_="listWrap relatedList").find_all("li"):
        recommend = i.find("a")
        recommend = recommend.find("h3").get_text()
        rec_list.append(recommend)
    recommends.append(rec_list)
    
    #pdf파일명 저장
    file = soup.find("dl", class_="attaFile")
    pdftitle = file.find("a").get_text()
    pdftitles.append(pdftitle)

    # PDF 다운로드 버튼 요소 찾기
    #pdf_button = driver.find_element_by_class_name("attaFile")
    #pdf_button.click()
    pdf.append('다운완료')

AttributeError: 'NoneType' object has no attribute 'find'

In [10]:
import pandas as pd

pdf_df = pd.DataFrame({"title":titles,
                       "recommend_list":recommends,
                       "pdf_title":pdftitles,
                       "download":pdf})
pdf_df

,title,recommend_list,pdf_title,download
0,인공지능 시대 새로운 성장 동력 HBM(고대역폭메모리),"[발전기 없는 발전소, 가상발전소(VPP), 해상풍력, 풍력발전의 새로운 도전, 스...",KB지식비타민_인공지능 시대 새로운 성장 동력 HBM_2307_홈페이지 등재용.pdf,다운완료
1,인공지능은 인간지능을 초월할 수 있을까,"[해외 대형은행의 온라인 대출중개 플랫폼 활용과 대응 사례, 트래블룰 적용 의의와 ...",(지식비타민) 인공지능은 인간지능을 초월할 수 있을까_최종.pdf,다운완료
2,"발전기 없는 발전소, 가상발전소(VPP)","[인공지능 시대 새로운 성장 동력 HBM(고대역폭메모리), 해상풍력, 풍력발전의 새...",(지식비타민)발전기 없는 발전소 가상발전소(VPP)_웹.pdf,다운완료
3,해외 대형은행의 온라인 대출중개 플랫폼 활용과 대응 사례,"[인공지능은 인간지능을 초월할 수 있을까, 트래블룰 적용 의의와 국내 가상자산 업계...",KB지식비타민_해외 대형은행의 온라인 대출중개 플랫폼 활용과 대응 사례.pdf,다운완료
4,트래블룰 적용 의의와 국내 가상자산 업계 동향,"[인공지능은 인간지능을 초월할 수 있을까, 해외 대형은행의 온라인 대출중개 플랫폼 ...",KB지식비타민_트래블룰 시행의 의의와 국내 가상자산업계 동향.pdf,다운완료
...,...,...,...,...
332,영국의 ISA 확장과 라이프타임 ISA 도입,"[중국 그린본드 시장 현황과 향후 전망, 100년 기업 육성을 위한 일본 정부와 은...",KB지식비타민_20160926_영국의 ISA 확장과 라이프타임 ISA 도입.pdf,다운완료
333,서비스업의 동적(Dynamic)가격 전략과 금융업 적용사례,"[일 잘하는 소비자, 팬슈머에 응답하라, 할매니얼들 윤며들다! : MZ세대를 감동시...",(2016-71)KB지식비타민_20160919_서비스업의 동적(Dynamic)가격 ...,다운완료
334,펫신탁의 개념과 일본 금융권 활용사례,"[화상통화를 활용한 비대면 보험 모집 해외 사례와 시사점, 아트테크(Art+재테크)...",KB지식비타민 16-70호_펫신탁의 개념과 일본 금융권 활용 사례_등재용.pdf,다운완료
335,전기자동차가 견인하는 전지 시장의 변화,"[인공지능 시대 새로운 성장 동력 HBM(고대역폭메모리), 발전기 없는 발전소, 가...",KB지식비타민_전가자동차가 견인하는 전지 시장의 변화_a.pdf,다운완료


In [11]:
pdf_df.to_csv('pdf테이블.csv')